## Семинар 8: "Современные модели для NLP"

ФИО: Калашников Дмитрий

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~150MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [1]:
import torch
!pip install --upgrade transformers
#!pip install sentencepiece
#from transformers import *

     |████████████████████████████████| 2.3MB 8.3MB/s 
     |████████████████████████████████| 3.3MB 33.9MB/s 
     |████████████████████████████████| 901kB 30.5MB/s 


In [2]:
from transformers import MobileBertForMaskedLM, MobileBertTokenizer

In [3]:
MODEL = (MobileBertForMaskedLM, MobileBertTokenizer, 'google/mobilebert-uncased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 2182, 2003, 2070, 3793, 2000, 4372, 16044, 102]


In [ ]:
tokenizer.decode(input_ids)

'[CLS] here is some text to encode [SEP]'

In [ ]:
input_ids = input_ids[:-1] + [tokenizer.mask_token_id] + [input_ids[-1]]
tokenizer.decode(input_ids)

'[CLS] here is some text to encode [MASK] [SEP]'

In [ ]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [ ]:
res.shape

torch.Size([1, 10, 30522])

In [ ]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [ ]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some text to encode..'

In [5]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

Сэмплирование с заданными вероятностями:

In [23]:
import random
import numpy as np
prob = [0] * 5
n = 10000
for i in range(n):
    t = random.choices([0,1,2,3,4], weights=[0.1,0.1,0.5,0.2,0.1], k=1)[0]
    prob[t] += 1
print(np.array(prob) / n)

[0.0972 0.0997 0.4967 0.2077 0.0987]


- Генерация выбором самого вероятного токена:

In [70]:
ans = []
MAX_TOKENS_COUNT = 60

for text in GPT_TEXTS:

    with torch.no_grad():
        input_ids = tokenizer.encode(' '.join( (text, '[MASK]' ) ), add_special_tokens=True) 
        input_ids = input_ids[:-1]
        for j in range(MAX_TOKENS_COUNT):

            input_batch = torch.tensor(input_ids).unsqueeze(0)
            res = model(input_batch)[0]
            prob = torch.nn.functional.softmax(res, dim=-1)
            input_ids = prob.max(-1)[1][0]

            input_ids = input_ids.tolist() + [tokenizer.mask_token_id] #, tokenizer.sep_token_id]

    text = tokenizer.decode(input_ids[:-1])
    if text[:2] == '. ':
        text = text[2:]

    text = text[::-1].split('.', 1)[1][::-1] + '.'
    ans.append( text )
                
print(ans[0], '\n', ans[1], '\n')

in a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. they also discovered a herd of unicorns living in a remote valley, in the andes mountains. they found a herd of unicorns living in a remote valley, in the andes mountains. scientists and researchers were surprised to find a herd of unicorns living in a remote valley, in the andes mountains. 
 a train carriage containing controlled nuclear materials was stolen in cincinnati today. its whereabouts are unknown. the train carriage was stolen in cincinnati today. it is unknown. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. the train carriage was stolen in cincinnati today. 



In [71]:
len(ans[0])

523

- Генерация сэмплированием из предсказанного распределения:

In [77]:
ans = []
MAX_TOKENS_COUNT = 60

for text in GPT_TEXTS:

    with torch.no_grad():
        input_ids = tokenizer.encode(' '.join( (text, '[MASK]' ) ), add_special_tokens=True) 
        input_ids = input_ids[:-1]
        for j in range(MAX_TOKENS_COUNT):

            input_batch = torch.tensor(input_ids).unsqueeze(0)
            res = model(input_batch)[0]

            prob = torch.nn.functional.softmax(res, dim=-1)

            input_ids = prob.max(-1)[1][0][:-1]
            
            last_ind = random.choices(range(prob.shape[-1]), weights=prob[0][-1], k=1)[0]

            input_ids = input_ids.tolist() + [last_ind, tokenizer.mask_token_id]

            # print(last_ind)
            # print(prob[0][-1][last_ind])
            # print(prob.max(-1)[0][0][-1])
            # print()

            #input_ids = input_ids.tolist() + [tokenizer.mask_token_id] #, tokenizer.sep_token_id]

    text = tokenizer.decode(input_ids[:-1])
    if text[:2] == '. ':
        text = text[2:]

    text = text[::-1].split('.', 1)[1][::-1] + '.'
    ans.append( text )
                
print(ans[0], '\n', ans[1], '\n')

in a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. the unicorn was the unicorn ’ s twin. the unicorn was his brother. perri was his. he now apparently was the master. he did that only to the humans and the others. you can see, simon goes to the numbers. 
 a train carriage containing nuclear nuclear materials was located in cincinnati today. its whereabouts are unknown. this was an abandoned train carriage and a part of the - - rail's - - - - - a train carriage containing nuclear nuclear materials, and energy efficient reactor stations at central park. the original building in rochester. with other buildings in rochester. fire and water site. 



- Run 1

in a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. the unicorn was the unicorn ’ s twin. the unicorn was his brother. perri was his. he now apparently was the master. he did that only to the humans and the others. you can see, simon goes to the numbers. 

 a train carriage containing nuclear nuclear materials was located in cincinnati today. its whereabouts are unknown. this was an abandoned train carriage and a part of the - - rail's - - - - - a train carriage containing nuclear nuclear materials, and energy efficient reactor stations at central park. the original building in rochester. with other buildings in rochester. fire and water site. 

- Run 2

in a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. the scientists trained hunters and joined the group when they left the research group at 8 : 38 am on 14 - 22 -, 2004 / 5 pm est. igbak province, japan, 8 : 00 am, wednesday, 19 - 02 - 2004. zaizz, fortifying the forest. 

 a train carriage containing controlled nuclear materials was stolen in cincinnati today. its whereabouts are unknown. one of the car is lost. it remains the only one on the railroad cars. as there was a different version of some cars in the railroad cars, so then there was just one car and one car. not one car there. 


## Дополнительно: сгенерируем с помощью GPT-2

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
!pip install simpletransformers
from simpletransformers.language_generation import LanguageGenerationModel

model = LanguageGenerationModel("gpt2", "gpt2", args={"max_length": 150}) # 1000

In [ ]:
for text in GPT_TEXTS:
    text = model.generate(text, verbose=False)
    text = '.'.join(text[0].split('.')[:-1]) + '.'
    print('#' * 40)
    print(len(text))
    print(text)
    print('#' * 40)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


########################################
881
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

Although it took centuries for the unicorns to speak any of the language spoken by humans on earth, the team says they finally decided to use these speakers to solve a problem: how to communicate with the extinct unicorns. The researchers used a radio telescope to map out the valley, with a pair of spectrophotometers on hand to help determine which unicorns were living in the area.

While most of the unicorns in the area speak the original German or Scandinavian languages, most are from China and Laos, and the Chinese language is the only one that's spoken by all the species. As the researchers say, many of the speakers do speak some German.
########################################
################################